In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_dataset, load_dataset_with_lables
from data_parser import get_tags_and_labels, get_vocab, get_tags

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# Get dataframe with tags and category_id columns
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)

# Creating different coloumn based on different tags
new = tab_dataframe["tags"].str.split(".", expand = True)

max_of_tags = 15

for i in range(0, max_of_tags):
    name = "tags"+str(i)
    tab_dataframe[name] = new[i]

# Dropping old tags columns 
tab_dataframe.drop(columns =["tags"], inplace = True) 
tab_dataframe.fillna("notags", inplace = True) 


tab_dataframe

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,10,Eminem,Walk,On,Water,Aftermath/Shady/Interscope,Rap,notags,notags,notags,notags,notags,notags,notags,notags,notags
1,23,plush,bad unboxing,unboxing,fan mail,idubbbztv,idubbbztv2,things,best,packages,plushies,chontent chop,notags,notags,notags,notags
2,23,racist superman,rudy,mancuso,king,bach,racist,superman,love,rudy mancuso poo bear black white official mus...,iphone x by pineapple,lelepons,hannahstocking,rudymancuso,inanna,anwar
3,24,ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,dares,no truth,comments,comedy,funny,stupid,fail,notags
4,10,edsheeran,ed sheeran,acoustic,live,cover,official,remix,official video,lyrics,session,notags,notags,notags,notags,notags
5,25,#DramaAlert,Drama,Alert,DramaAlert,keemstar,youtube news,jake paul,team 10,alissa violet,cheated,logan paul,logan paul alissa violet,jake paul alissa violet,Martinez Twins,left team 10
6,23,Funny Moments,Montage video games,gaming,Vanoss,VanossGaming,Gameplay,Secret,Puncake,Parody,Comedy,Remix,Epic,Trolling,Xbox One,MultiplayerPS4
7,22,SHANtell martin,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags
8,24,logan paul vlog,logan paul,logan,paul,olympics,logan paul youtube,vlog,daily,comedy,hollywood,parrot,maverick,bird,maverick clothes,logan paul dubai
9,22,God,Sheldon Cooper,Young Sheldon,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags,notags


In [18]:
# Getting leaning vocab
vocab = get_vocab(get_tags(csvpath))
voc_di = {i:vo for i,vo in enumerate(vocab)}
voc_di[0] = 'notags'
voc_di = {vo:i for i, vo in voc_di.items()}

TypeError: 'dict_items' object is not subscriptable

In [ ]:
# Mapping tags to indexes in vocab
for index in range(0, max_of_tags):
    tab_dataframe['tags{}'.format(index)] = tab_dataframe['tags{}'.format(index)].map(voc_di)
    
tab_dataframe

In [ ]:
for column in tab_dataframe.columns:
    if column == 'category_id':
        pass
    else:
        if tab_dataframe[column].max() == 0:
            tab_dataframe[column] = tab_dataframe[column]

        else:
            tab_dataframe[column] = tab_dataframe[column] / tab_dataframe[column].max()

tab_dataframe

In [ ]:
# Create train,validation and test examples from dataframe
# train_ds, val_ds, test_ds = load_dataset(tab_dataframe)
train_ds, test_ds, train_lb, test_lb = load_dataset_with_lables(tab_dataframe, 'category_id')


In [ ]:
print(train_ds)

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.01), input_shape = (32, 25)),
    layers.Dropout(0.5),
    layers.Dense(128, kernel_regularizer=keras.regularizers.l2(0.01), input_shape = (32, 64)),
    layers.Dense(64, activation='sigmoid', input_shape = (32, 128)),
    layers.Dense(44, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.01), input_shape = (32, 64))
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'binary_crossentropy'],
              )

history = model.fit(train_ds, train_lb,
          batch_size=32,
          epochs=5)
